### Runtime setup
The following envs enable stable retries and quiet streaming.

- `SCILLM_FORCE_HTTPX_STREAM=1`
- `LITELLM_MAX_RETRIES=3`, `LITELLM_RETRY_AFTER=1`, `LITELLM_TIMEOUT=45`
- Requires `tenacity` installed for backoff.

In [ ]:
import os
os.environ.setdefault('SCILLM_FORCE_HTTPX_STREAM','1')
os.environ.setdefault('LITELLM_MAX_RETRIES','3')
os.environ.setdefault('LITELLM_RETRY_AFTER','1')
os.environ.setdefault('LITELLM_TIMEOUT','45')
try:
    import tenacity  # noqa: F401
    print('tenacity: ok')
except Exception:
    print('tenacity missing — run: pip install tenacity')


# Chutes — OpenAI-Compatible

        Minimal chat using the OpenAI-compatible path.

In [ ]:
import os, litellm
resp = litellm.completion(
  model=os.environ['CHUTES_MODEL'],
  api_base=os.environ['CHUTES_API_BASE'],
  api_key=None,
  custom_llm_provider='openai_like',
  # JSON paved path: use x-api-key (no Bearer)
  extra_headers={'x-api-key': os.environ['CHUTES_API_KEY']},
  messages=[{'role':'user','content':'Say OK'}],
  max_tokens=8,
  temperature=0,
)
print(resp.choices[0].message.get('content',''))